In [1]:
! unzip archive.zip -d ./csv/

Archive:  archive.zip
  inflating: ./csv/governors_county.csv  
  inflating: ./csv/governors_county_candidate.csv  
  inflating: ./csv/governors_state.csv  
  inflating: ./csv/house_candidate.csv  
  inflating: ./csv/house_state.csv   
  inflating: ./csv/president_county.csv  
  inflating: ./csv/president_county_candidate.csv  
  inflating: ./csv/president_state.csv  
  inflating: ./csv/senate_county.csv  
  inflating: ./csv/senate_county_candidate.csv  
  inflating: ./csv/senate_state.csv  
  inflating: ./csv/world_country_and_usa_states_latitude_and_longitude_values.csv  


# The CRISP-DM Process

The 59th quadrennial US presidential election was held on Tuesday, November 3, 2020. To win the election, the candidate needs as a minimum 270 out of 538 electoral votes.

The dataset was obtained from Kaggle: https://www.kaggle.com/unanimad/us-election-2020

## Business Understanding
Pose at least three questions related to business or real-world applications of how the data could be used.

1. Which state had the absolute highest number of votes placed in the presidential election?
2. Which states were won by which candidate in the presidential election? 
3. What is the distribution of non-major party votes per state for the presidential election?

## Data Understanding

* Question #1
    
    For this question, president_state.csv has state level number of votes. This data can be further coupled with state decsriptions for meaningful illustrations.

* Question #2

    The pres_cty_cand.csv file has county level candidate votes, together with party affiliation, which can be also joined with state descriptions for meaningful results.

* Question #3

    Same as for Q#2, howeer this time the focus is on non-major, "third-party" candidates.


Categorical values were found to be useful in this case, as most of them represent labels which can be used for filtering and selecting table subsets. Furthermore, categorical values were also encoded into columns to represent vote counts from different parties.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

## Data preparation & modelling

In [3]:
# gather the datasets related to the presidential election
df_pres_cty_cand = pd.read_csv("./csv/president_county_candidate.csv")
df_pres_state = pd.read_csv("./csv/president_state.csv")
df_pres_cty = pd.read_csv("./csv/president_county.csv")

# read auxiliary dataset about US geographical information to be used later
df_us_state_codes = pd.read_csv("./csv/world_country_and_usa_states_latitude_and_longitude_values.csv")

#### Assess the election tables

In [4]:
print(df_pres_cty_cand.columns)
df_pres_cty_cand.head()

Index(['state', 'county', 'candidate', 'party', 'total_votes', 'won'], dtype='object')


,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


In [5]:
print(df_pres_state.columns)
df_pres_state.head()

Index(['state', 'total_votes'], dtype='object')


,state,total_votes
0,Delaware,504010
1,District of Columbia,344356
2,Florida,11067456
3,Georgia,4997716
4,Hawaii,574469


In [6]:
print(df_pres_cty.columns)
df_pres_cty.head()

Index(['state', 'county', 'current_votes', 'total_votes', 'percent'], dtype='object')


,state,county,current_votes,total_votes,percent
0,Delaware,Kent County,87025,87025,100
1,Delaware,New Castle County,287633,287633,100
2,Delaware,Sussex County,129352,129352,100
3,District of Columbia,District of Columbia,41681,41681,100
4,District of Columbia,Ward 2,32881,32881,100


In [7]:
print(df_us_state_codes.columns)
df_us_state_codes.head()

Index(['country_code', 'latitude', 'longitude', 'country', 'usa_state_code',
       'usa_state_latitude', 'usa_state_longitude', 'usa_state'],
      dtype='object')


,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas
3,AG,17.060816,-61.796428,Antigua and Barbuda,AZ,34.048928,-111.093731,Arizona
4,AI,18.220554,-63.068615,Anguilla,CA,36.778261,-119.417932,California


In [8]:
df_us_state_codes = df_us_state_codes[["usa_state_code", "usa_state"]]  # get only the required columns (state codes and states)

In [9]:
df_us_pres_state_votes = df_pres_state.merge(df_us_state_codes, left_on="state", right_on="usa_state")
df_us_pres_state_votes = df_us_pres_state_votes.drop(["usa_state"], axis=1)
df_us_pres_state_votes.head()

,state,total_votes,usa_state_code
0,Delaware,504010,DE
1,District of Columbia,344356,DC
2,Florida,11067456,FL
3,Georgia,4997716,GA
4,Hawaii,574469,HI


## 1. Which state had the absolute highest number of votes placed in the presidential election?

In [10]:
fig = px.choropleth(df_us_pres_state_votes, locations="usa_state_code", color="total_votes", 
                    range_color=(0, 10000000),
                    locationmode = "USA-states",
                    scope="usa",
                    title="Presidential vote count per state")
fig.show()

In [11]:
df_us_pres_state_votes.sort_values(by=["total_votes"], ascending=False)[:5] # sort the new dataframe by number of votes and show the top 5

,state,total_votes,usa_state_code
47,California,17495906,CA
36,Texas,11317911,TX
2,Florida,11067456,FL
25,New York,8616205,NY
31,Pennsylvania,6925255,PA


#### Answer 
* Based on the above table and plot, California is the state with the highest amount of votes by a comfortable 6 million lead.

## 2. Which states were won by which candidate in the presidential election?

In [12]:
#referennce : https://www.kaggle.com/paultimothymooney/2020-usa-election-vote-percentages-by-state

df_pres_cty_cand_main = df_pres_cty_cand[(df_pres_cty_cand["party"] == "DEM") | (df_pres_cty_cand["party"] == "REP")]  # get all the candidates which are either DEM or REP
print(df_pres_cty_cand_main.shape)
df_pres_cty_cand_main.head()

(9266, 6)


,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True
5,Delaware,New Castle County,Donald Trump,REP,88364,False
8,Delaware,Sussex County,Donald Trump,REP,71230,True


In [13]:
# calculate the total vote sum for each party in each state. The highest number represents which party won which state (and it's corresponding electoral votes).

df_pres_cty_cand_main = df_pres_cty_cand_main.groupby(["state", "party"])["total_votes"].sum()   
df_pres_cty_cand_main.columns = ["DEM", "REP"]
df_pres_cty_cand_main = df_pres_cty_cand_main.to_frame().reset_index()
df_pres_cty_cand_main.head()

,state,party,total_votes
0,Alabama,DEM,849648
1,Alabama,REP,1441168
2,Alaska,DEM,153405
3,Alaska,REP,189892
4,Arizona,DEM,1672143


In [14]:
# create temporary dataframe with only the states, and sort them alphabetically

temp = df_pres_cty_cand_main.drop_duplicates(subset=["state"]).sort_values("state").reset_index(drop=True)
temp = temp["state"].to_frame()

In [15]:
# get the votes for each party in every state and make them individual columns

dem_votes = df_pres_cty_cand_main[df_pres_cty_cand_main["party"] == "DEM"]["total_votes"].to_frame().reset_index(drop=True)  # for the Dem party
rep_votes = df_pres_cty_cand_main[df_pres_cty_cand_main["party"] == "REP"]["total_votes"].to_frame().reset_index(drop=True)  # for the Rep party
df_pres_cty_cand_main = pd.concat([temp, dem_votes, rep_votes], axis=1)  # join the vote count columns with the state name column
df_pres_cty_cand_main.columns = ["state", "DEM", "REP"]  # rename the columns 

In [16]:
# merge the state & party vote counts table with the state codes table for visualization

df_pres_cty_cand_main = df_pres_cty_cand_main.merge(df_us_state_codes, left_on="state", right_on="usa_state")
df_pres_cty_cand_main = df_pres_cty_cand_main.drop(["usa_state"], axis=1)

In [17]:
df_pres_cty_cand_main.head()

,state,DEM,REP,usa_state_code
0,Alabama,849648,1441168,AL
1,Alaska,153405,189892,AK
2,Arizona,1672143,1661686,AZ
3,Arkansas,423932,760647,AR
4,California,11109764,6005961,CA


In [18]:
# calculate the percentage of DEM votes. Since there are only two possibilities, 1-%_dem gives the percentage of REP votes. More than 50% is needed.

df_pres_cty_cand_main["%_dem"] = df_pres_cty_cand_main['DEM']*100/(df_pres_cty_cand_main['REP']+df_pres_cty_cand_main['DEM'])   # multiply by 100 to also scale the numbers
df_pres_cty_cand_main.head()

,state,DEM,REP,usa_state_code,%_dem
0,Alabama,849648,1441168,AL,37.089317
1,Alaska,153405,189892,AK,44.685797
2,Arizona,1672143,1661686,AZ,50.156832
3,Arkansas,423932,760647,AR,35.787567
4,California,11109764,6005961,CA,64.909690


In [19]:
fig = px.choropleth(df_pres_cty_cand_main, 
                    locations="usa_state_code", 
                    color = "%_dem",
                    locationmode = 'USA-states', 
                    hover_name="state",
                    range_color=[25, 75],
                    color_continuous_scale = 'RdBu',
                    scope="usa",
                    title='Percentage of Population Voting for the Democratic candidate')
fig.show()

#### Answer 
* Based on the above plot, states with the color blue (all shades) were won by the Democratic Party candidate. States with the color red (all shades) were won by the Republican Party candidate. The darker the shade the bigger the lead for the corresponding party.

## 3. Which non-major party had the highest amount of votes in each state?

In [20]:
# get only the candidates from other parties (non DEM or REP)

df_pres_cty_cand_third = df_pres_cty_cand[(df_pres_cty_cand["party"] != "DEM") & (df_pres_cty_cand["party"] != "REP") & (df_pres_cty_cand["party"] != "WRI")]

In [21]:
# calculate the total vote sum for each party in each state.

df_pres_cty_cand_third = df_pres_cty_cand_third.groupby(["state", "party"])["total_votes"].sum().to_frame()
df_pres_cty_cand_third.head(5)

total_votes
state   party             
Alabama LIB          25176
Alaska  ALI            318
        CST           1127
        GRN           2673
        IND            825

In [22]:
# get the "third-party" with the maximum votes for each state and save them in the table

max_idx = df_pres_cty_cand_third.groupby(["state"])["total_votes"].transform(max) == df_pres_cty_cand_third["total_votes"]
df_pres_cty_cand_third = df_pres_cty_cand_third[max_idx]
df_pres_cty_cand_third = df_pres_cty_cand_third.reset_index()
df_pres_cty_cand_third.head()

,state,party,total_votes
0,Alabama,LIB,25176
1,Alaska,LIB,8896
2,Arizona,LIB,51465
3,Arkansas,LIB,13133
4,California,LIB,187885


In [23]:
# merge the above table with most "third-party" votes with the original US states and state codes table

df_pres_cty_cand_third = df_pres_cty_cand_third.merge(df_pres_state, left_on="state", right_on="state")
df_pres_cty_cand_third.columns = ["state", "party", "total_party_votes", "total_votes"]  # rename the columns
df_pres_cty_cand_third["%_party_votes"] = df_pres_cty_cand_third["total_party_votes"] * 100 / df_pres_cty_cand_third["total_votes"]   # calculate the percentage of votes taken by the "third-party" with the most amount of votes in each state
df_pres_cty_cand_third["state_code"] = df_pres_cty_cand_main["usa_state_code"]
df_pres_cty_cand_third.head()

,state,party,total_party_votes,total_votes,%_party_votes,state_code
0,Alabama,LIB,25176,2323304,1.083629,AL
1,Alaska,LIB,8896,359530,2.474342,AK
2,Arizona,LIB,51465,3387326,1.519340,AZ
3,Arkansas,LIB,13133,1219069,1.077298,AR
4,California,LIB,187885,17495906,1.073880,CA


In [24]:
fig = px.choropleth(df_pres_cty_cand_third, 
                    locations="state_code", 
                    color = "%_party_votes",
                    locationmode = 'USA-states', 
                    hover_name="state",
                    range_color=[0.5, 3],
                    color_continuous_scale = 'ylorbr',
                    scope="usa",
                    title='Percentage of third-party votes in every state')
fig.show()

In [25]:
df_pres_cty_cand_third.sort_values(by=["%_party_votes"], ascending=False)[:5] # sort the new dataframe by % of votes and show the top 5

,state,party,total_party_votes,total_votes,%_party_votes,state_code
41,South Dakota,LIB,11095,422609,2.625358,SD
34,North Dakota,LIB,9393,361819,2.596049,ND
44,Utah,LIB,38447,1488289,2.583302,UT
26,Montana,LIB,15252,605750,2.517870,MT
1,Alaska,LIB,8896,359530,2.474342,AK


#### Answer 
* Based on the above table, although there are a number of non-major parties involved in the presidential election, the highest voted "third-party" in every state is the Libertarian Party, with the most votes taken in South Dakota (11095, 2.625 %), followed by North Dakota and Utah.

* The interactive plot illustrates this, with states that have a darker shade of orange having a higher number of votes